# Homework 1 - Image Classification
In this notebook, we have used Transfer Learning in order to achieve our best score on Kaggle. We used a **DenseNet201**, to which we added a **BatchNormalization** layer just before the output dense layer. We experimented also the insertion of the **ReduceLearningRateOnPlateau** callback function, which led us to better results once the learning started to stagnates.

In [1]:
#import libraries
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import numpy as np
import tensorflow as tf
import json
import pandas as pd

In [2]:
#set random seed
SEED = 1234
tf.random.set_seed(SEED)  
cwd = os.getcwd()

In [ ]:
#Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#unzip challenge folders
!unzip '/content/drive/My Drive/artificial-neural-networks-and-deep-learning-2020.zip'

In [5]:
#dataset directories
dataset_dir = os.path.join(cwd, 'MaskDataset')

training_dir = os.path.join(dataset_dir, 'training')

test_dir = os.path.join(dataset_dir, 'test')

In [6]:
#ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#DataAugmentation

apply_data_augmentation = True

if apply_data_augmentation:
  train_data_gen = ImageDataGenerator(rotation_range=30, 
                                      width_shift_range=30, 
                                      height_shift_range=30, 
                                      zoom_range=0.6, 
                                      horizontal_flip=True, 
                                      vertical_flip=True,
                                      fill_mode='constant',
                                      cval=0,
                                      rescale=1/255.,
                                      validation_split=0.2)
else:
  train_data_gen = ImageDataGenerator(rescale=1/255.,validation_split=0.2)

valid_data_gen = ImageDataGenerator(rescale=1/255.)

test_data_gen = ImageDataGenerator(rescale=1/255.)

In [7]:
#param. setting

#BatchSize
bs = 16 #bs = 32 affected negatively the performances of our net

#ImageSizes for DenseNet201
img_h = 224
img_w = 224

#NumClasses
num_classes = 3

In [ ]:
#DataGenerators
with open('MaskDataset/train_gt.json') as f:
  dic = json.load(f)
dataframe = pd.DataFrame(dic.items())
dataframe.rename(columns = {0:'filename', 1:'class'}, inplace = True)
dataframe["class"] = dataframe["class"].astype(str)


train_gen = train_data_gen.flow_from_dataframe(dataframe,
                                               training_dir,
                                               batch_size=bs,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED,
                                               subset='training')

valid_gen = train_data_gen.flow_from_dataframe(dataframe,
                                               training_dir,
                                               batch_size=bs,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED,
                                               subset='validation')

test_gen = test_data_gen.flow_from_directory(dataset_dir, classes=['test'], batch_size=bs, class_mode=None,shuffle=False, seed=SEED)
valid_gen

In [9]:
#Datasets
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen, output_types=(tf.float32, tf.float32), output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
train_dataset = train_dataset.repeat()

valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, output_types=(tf.float32, tf.float32), output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
valid_dataset = valid_dataset.repeat()

test_dataset = tf.data.Dataset.from_generator(lambda: test_gen, output_types=(tf.float32, tf.float32), output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

In [ ]:
#VisualizeTrainSet
import matplotlib.pyplot as plt
%matplotlib inline
TrainIterator = iter(train_dataset)
train_augmented_img, target = next(TrainIterator)
train_augmented_img = np.array(train_augmented_img[0])
train_augmented_img = train_augmented_img * 255
plt.imshow(np.uint8(train_augmented_img))
plt.plot()

In [ ]:
#VisualizeValidSet
import matplotlib.pyplot as plt
ValidIterator = iter(valid_dataset)
valid_augmented_img, target = next(ValidIterator)
valid_augmented_img = np.array(valid_augmented_img[1])
valid_augmented_img = valid_augmented_img*255
plt.imshow(np.uint8(valid_augmented_img))
plt.plot()

In [10]:
#NeuralNetwork
denseNet201 = tf.keras.applications.DenseNet201(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
)

model = tf.keras.Sequential()
model.add(denseNet201)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=512, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [ ]:
model.summary()

In [11]:
#Loss, LR,and Metrics
loss = tf.keras.losses.CategoricalCrossentropy()

lr = 1e-4 #initially we tried lr = 1e-5, but with RLROP function, 1e-4 is better (faster but equally precise)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

metrics = ['accuracy']

In [12]:
#Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [14]:
import os
from datetime import datetime


cwd = os.getcwd()

exps_dir = os.path.join('/content/drive/My Drive/Keras3/', 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint

ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard

tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping

early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    callbacks.append(es_callback)

#ReduceLRonPlateau
#This function led us to better results
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor= 'val_loss', factor = 0.1, patience=5, verbose = 1, min_lr = 1e-5)

callbacks.append(reduce_lr)

In [ ]:
#Train the Model
model.fit(x=train_dataset, epochs=100, steps_per_epoch=len(train_gen), validation_data=valid_dataset, validation_steps=len(valid_gen), callbacks=callbacks)

In [ ]:
#Test Prediction
STEP_SIZE_TEST = len(test_gen)
pred=model.predict(test_gen,steps=STEP_SIZE_TEST,verbose=1)

predictions = []

for i in pred:
    if i[0]>i[1] and i[0]> i[2]:
        predictions.append('0')
    elif i[1] > i[2]:
        predictions.append('1')
    else:
        predictions.append('2')

In [17]:
#Create the results
filenames=test_gen.filenames
for i in range(0,len(filenames)):
  filenames[i] = filenames[i][5:]

results=pd.DataFrame({"Id":filenames,
                      "Category":predictions})

In [ ]:
results

In [20]:
#Create the csv file
results.to_csv('/content/drive/My Drive/Keras3/out20.csv', index=False) 